# Esercizio 3 - SemEval

## Consegna

#### Parte 1

1. **Annotazione:** Annotare il file con i valori di similarità dati da noi e con i `BabelNetSynsetID` scelti da noi dal file `SemEval17_IT_senses2synsets.txt`.
   
2. **Agreement:** Dopo di che si calcola il `valore medio` di similarità, `Pearson` e `Spearman`
   
3. **Ricerca Automatica del Synset:** Andiamo a cercare il `BabelNetSynsetID` nel file dato dal prof `mini_NASARI.tsv`
   andando a massimizzare la *cosine similarity*. I valori che massimizzeranno quest'ultima saranno i nostri concetti (Synset).

   Per far ciò andiamo ad estrarre tutti i synsets relativi ad un termine, dal file `SemEval17_IT_senses2synsets.txt`
   e andiamo a calcolare la *cosine similarity* per ogni synset (quelli dei due termini) presente nel file `mini_NASARI.tsv`
   e salviamo i due sysnets che massimizzano la *cosine similarity*.

4. **Valutazione Similarità:** Sempre con `Pearson` e `Spearman`, tra i nostri valori di similarità e quelli trovati
   al passo precedente.

#### Parte 2

1. Associare i synset ai termini che pensiamo che massimizzino la similarità (fatto a mano) --> Gia fatto al passo 1 

2. Output costituito da 6 campi: 
   1. Term1
   2. Term2
   3. BS1 (dato da noi)
   4. BS2 (dato da noi)
   5. Sinonimi Term1 (da synset a lemma)
   6. Sinonimi Term2 (da synset a lemma)

   <br/>     
3. **Agreement dell'anotazione Synset trovati:** Tramite il punteggio `kappa di Cohen` calcoliamo il livello 
   di agreement (sklearn) tra i synset trovati da noi

4. **Valutazione annotazione:** Confronto i nostri concetti (synset) con quelli che massimizzano la `cos_sim`
   (Trovati già al passo 3)


# Sviluppo

#### Imports

In [1]:
import hashlib
import matplotlib.pyplot as plt

import re
import csv

from scipy import spatial

import numpy as np

import requests

from sklearn.metrics import cohen_kappa_score

## Parte 1

### 1, 2 - Annotazione e Agreement

#### Organizzazione dei dati

Andiamo a selezionare il dati da utilizzare per l'esercizio 3 tramite la funzione data dal professore

In [2]:
path_data = "../data/es3_res/it.test.data.txt"
annotator = "Bonicco"
data_file = "SemEval_data_" + annotator + ".tsv"

In [3]:
def get_range(surname):
    nof_elements = 500
    base_idx = (abs(int(hashlib.sha512(surname.encode('utf-8')).hexdigest(), 16)) % 10)
    idx_intervallo = base_idx * 50+1
    return idx_intervallo
 

input_name = "Bonicco"

values = []
sx = get_range(input_name)
values.append(sx)
dx = sx+50-1
intervallo = "" + str(sx) + "-" + str(dx)
print('{:15}:\tcoppie nell\'intervallo {}'.format(input_name, intervallo))

Bonicco        :	coppie nell'intervallo 51-100


##### Creazione file con le coppie di termini che ci interessano - **Non eseguire, file già creato**

In [4]:
with open(path_data) as file:
    lines = file.readlines()[50:100]
    lines = [line.rstrip() for line in lines]
    print(lines)

['biotopo\tbiologia', 'magma\tvulcano', 'brainstorming\ttelescopio', 'livello\tpunteggio', 'centesimo\taffare', 'partito politico\tassociazione', 'tsunami\tmare', 'struzzo\tfrutteto', 'cannella\tcaramella', 'scopa\tpolvere', 'galassia\tastronomo', 'succo\tfrappè', 'tapparella\ttenda', 'criminale\tcolpevole', 'cancro al pancreas\tchemioterapia', 'passato\tantecedente', 'Nazioni Unite\tParlamento Europeo', 'canzone\tesecutore', 'pistola\ttaccuino', 'acetilcolina\tiride', 'alfabeto\tpenna', 'coro\tcantante', 'legno\tcoperta', 'soldato\tpace', 'inglese\tamericano', 'lucertola\tcoccodrillo', 'denaro\tcontante', 'Polpo Paul\tpolpo', 'calendario\tvacanza', 'base\tsostanza chimica', 'governo\tlucertola', 'JPY\triciclaggio di denaro', 'DeepMind\tGoogle DeepMind', 'scacchi\tscacco al re', 'sonetto\tbellezza', 'lavoratore\tufficio', 'valuta\tscambio', 'poliestere\tcotone', 'tribunale\tgiustizia', 'coda\tretro', 'incidente\tlibro', 'groppo\tvento', 'piano\tarmonia', 'sultano\tministero', 'deserto\

In [5]:
"""terms_sim_score = []

 with open(data_file, 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Term1", "Term2", "Similarity"])
    
    for tuple in lines:
        w_couple = re.split(r'\t+', tuple.rstrip('\t'))
        print(w_couple)
        #! Scommentare per annotazione "automatica"
        # score = input("\tInserisci il valore di similarità [0 4] per i * "+ str(w_couple) +" *:")
        
        tsv_writer.writerow(w_couple) """


'terms_sim_score = []\n\n with open(data_file, \'wt\') as out_file:\n    tsv_writer = csv.writer(out_file, delimiter=\'\t\')\n    tsv_writer.writerow(["Term1", "Term2", "Similarity"])\n    \n    for tuple in lines:\n        w_couple = re.split(r\'\t+\', tuple.rstrip(\'\t\'))\n        print(w_couple)\n        #! Scommentare per annotazione "automatica"\n        # score = input("\tInserisci il valore di similarità [0 4] per i * "+ str(w_couple) +" *:")\n        \n        tsv_writer.writerow(w_couple) '

#### Excel

Nel seguente file excel abbiamo annotato i punteggi di similarità relativi alla coppia di termini, dopo di che abbiamo calcolato la media tra i valori dati
dai due annotatori, oltre ai due indici di correlazione *Pearson* e *Spearman*.

Inoltre, abbiamo anche annotato i synset relativi ai termini, andandoli a scegliere tra la lista dei synset data dal professore.

<a href="https://1drv.ms/x/s!AkdbdBkdl_1dhbcgYl1tXDNpjknAIw?e=UYsHF5"> Link all'excel con i dati </a>

#### Calcolo Spearman per Excel

In [6]:
def pearson_index_np(x, y): # [3,45 ; 5,66 ; ..]
    '''
    Implementation of the Pearson index using numpy
    Args:
         x: golden value
         y: similarity list
    Returns:
        Pearson correlation index = [Covariance / (Standard deviation of x * Standard deviation of y)]
    '''
    mu_x = np.mean(x)
    mu_y = np.mean(y)
    
    # and the standard deviation of both
    std_dev_x = np.std(x)
    std_dev_y = np.std(y)
    
    numeric = std_dev_x * std_dev_y

    num = np.cov(x, y)[0][1] # Covariance    

    return num / numeric

def spearman_index(x, y):
    '''
    Implementation of the Spearman index.
    Args:
        x: golden value
        y: similarity list
    Returns:
         Spearman correlation index
    '''
    rank__x = define_rank(x)
    rank__y = define_rank(y)

    return pearson_index_np(rank__x, rank__y)


def define_rank(vector):
    '''
    Args:
        vector: numeric vector
    Returns:
        ranks list, sorted as the input order
    '''
    x_couple = [(vector[i], i) for i in range(len(vector))]
    x_couple_sorted = sorted(x_couple, key=lambda x: x[0])

    return [y for (x, y) in x_couple_sorted]

In [7]:
sim_paolo = [2,2.6,0,1.8,0.8,1.5,3,0,1,2.5,2.8,1,2.4,2.8,2.2,4,1.4,1.6,0,0.2,0.5,3,0,1.8,2,2.6,3,3,1,3,0,1.4,4,3.6,0.4,3,2.5,2,2.8,0.4,0,3,2.8,1.8,3.4,3.6,1.8,0,0,2.4]
sim_davide = [1.8,2,0,1.8,1.2,2.8,1,0,0.8,1,1,3,1.5,2.8,1.8,3.5,0.8,0,0,0.5,1,3,0.8,1,3.2,3,4,2.5,1,2,0,1.2,4,2,0,2,2,3,2,3.8,0,2,1.5,1.2,2,3,1.5,0,0,1.2]

print(len(sim_paolo) == len(sim_davide) == 50)

True


In [8]:
spearman_index(sim_paolo, sim_davide)

0.1991817135017272

In [9]:
# open(data_file, 'w').close() # Puliamo il file con i termini di riferimento

### 3 - Ricerca automatica del Synset - Massimizzazione cosine similarity da mini_NASARI

#### Lista di synset associati ad un termine

In [10]:
def cos_sim(vec1, vec2):
    '''
    Given 2 vectors, return the cosine similarity between them
    '''
    return 1 - spatial.distance.cosine(vec1, vec2)

def get_synsets_from_word(path, word):
    '''Given the path of the file with the list of synset and a word return the list of synset of the word
    Args:
        path: path of the file with the list of synset
        word: word to search in the file
    Returns:
        list of synset of the word
    '''
    syns_list = []
    with open(path) as f:
        terms_synsets = f.readlines()
        for i in range(len(terms_synsets)):
            if word in terms_synsets[i]:
                index = i
                while(terms_synsets[index+1].find('#') == -1): # and index < len(terms_synsets)): #! Funziona anche senza perchè non andiamo a prendere l'ultimo termine
                    syns_list.append(terms_synsets[index+1].rstrip('\n'))
                    index += 1
                break
    return syns_list


In [11]:
def parse_mini_NASARI(path):
    '''Given the path of the nasari file it return a dict with the synset as key and the vector as value
    Args:
        path: path of the nasari file
    Returns:
        A dict with the synset as key and the vector as value
    '''
    term_list = {}
    with open(path) as fd:
        rd = csv.reader(fd, delimiter="\t", quotechar='"')
        for row in rd:
            synset= row[0]
            index = synset.index('_')
            synset = synset[:index]
            term_list[synset] = row[1:]
            
    return term_list

def max_nasari_similarity(w1, w2):
    '''Given 2 word return the max value of similarity based on NASARI vector space model
    The method search for synset in nasari file and if find a value evaluate the similarity
    Args:
        w1: first word
        w2: second word
    Returns:
        max value of similarity
    '''
    path_synsets = "../data/es3_res/SemEval17_IT_senses2synsets.txt"
    path_nasari = "../data/es3_res/mini_NASARI.tsv"
    nasari = parse_mini_NASARI(path_nasari) # Dict with synset as key
    synsets_w1 = get_synsets_from_word(path_synsets, w1)
    synsets_w2 = get_synsets_from_word(path_synsets, w2)
    max_sim = [0, "", ""]
    for s1 in synsets_w1:
        if(s1 in nasari.keys()):
            for s2 in synsets_w2:
                if(s2 in nasari.keys()):
                    v1 = list(map(float, nasari[s1]))
                    v2 = list(map(float, nasari[s2]))
                    sim = cos_sim(v1, v2)
                    if sim > max_sim[0]:
                        max_sim = [sim, s1, s2]
    return max_sim

def couple_list(path):
    ''' Given a path return a list of couples of terms '''
    with open(path) as f:
        lines = f.readlines()
        couple_list = []
        for couple in lines:
            tmp = couple.split("\t")
            tmp[1] = tmp[1].rstrip('\n')
            couple_list.append(tmp)
        
        return couple_list

In [12]:
term_list_path = "resource/term_list.txt"

couples = couple_list(term_list_path)

In [13]:
values = []
for couple in couples:
    w1 = couple[0]
    w2 = couple[1]
    nasari_sim = max_nasari_similarity(w1, w2)
    # print("Similarity between " + w1 + " and " + w2 + " is " + str(nasari_sim[0]))
    # print(round(nasari_sim[0], 4))
    # print(nasari_sim[1] + "\t" + nasari_sim[2])
    values.append(round(nasari_sim[0], 4))

### 4 - Spearman tra media similarità annotatori e cosine similarity

In [14]:
cos_sim = values
#sim_mean = [1.9, 2.3, 0, 1.8, 1, 2.15, 2, 0, 0.9, 1.75, 1.9, 2, 1.95, 2.8, 2, 3.75, 1.1, 0.8, 0, 0.35, 0.75, 3, 0.4, 1.4, 2.6, 2.8, 3.5, 2.75, 1, 2.5, 0, 1.3, 4, 2.8, 0.2, 2.5, 2.25, 2.5, 2.4, 2.1, 0, 2.5, 2.15,1.5, 2.7, 3.3, 1.65, 0, 0, 1.8]
sim_norm_mean = [0.475,0.575,0,0.45,0.25,0.5375,0.5,0,0.225,0.4375,0.475,0.5,0.4875,0.7,0.5,0.9375,0.275,0.2,0,0.0875,0.1875,0.75,0.1,0.35,0.65,0.7,0.875,0.6875,0.25,0.625,0,0.325,1,0.7,0.05,0.625,0.5625,0.625,0.6,0.525,0,0.625,0.5375,0.375,0.675,0.825,0.4125,0,0,0.45]

round(spearman_index(cos_sim, sim_norm_mean), 4)

0.137

## Parte 2

### Estrazione dei sinonimi dei termini per comporre l'output:

Output costituito da 6 campi: 
   1. Term1
   2. Term2
   3. BS1 (dato da noi)
   4. BS2 (dato da noi)
   5. Sinonimi Term1 (da synset a lemma)
   6. Sinonimi Term2 (da synset a lemma)

In [96]:
#KEY = '8be6bdbd-78ee-4efb-91a2-3854f79a97e0'
KEY = '4ac4317e-cf96-4222-ba17-753d12dc7a2f'
URL = 'https://babelnet.io/v7/getSynset?id={}&key={}&targetLang=IT&searchLang=IT'
    
def get_synonims_for_synsets(babel_synsets_ids):
    '''
    Given a list of babelNetSynset IDs returns a dictionary containing,
    for each ID, the set of all lemmas in the corresponding BabelNet Synset.
    '''
    res = dict()
    for id in babel_synsets_ids:
        if id in res.keys(): 
            continue
        res[id] = set()
        x = requests.get(URL.format(id,KEY))
        if x.status_code == 400:
            res[id] = list(res[id])
            continue
        senses = x.json()['senses']
        for sense in senses:
            res[id].add(sense['properties']['fullLemma'])
        res[id] = list(res[id])
    return res

In [97]:
terms_1 = []
terms_2 = []
for el in couples:
    terms_1.append(el[0])
    terms_2.append(el[1])

Vado ad estrarre i synset dal file di testo

In [98]:
syn_list_path = "resource/synset_lists.txt"
syns = []
with open(syn_list_path) as f:
    lines = f.readlines()
    for line in lines:
        syns.append(line.strip("\n").split(","))

In [99]:
bs1_p = syns[0]
bs2_p = syns[1]

bs1_d = syns[2]
bs2_d = syns[3]

nasari_bs1 = syns[4]
nasari_bs2 = syns[5]

In [100]:
synsets = set(bs1_p).union(set(bs2_p), set(bs1_d), set(bs2_d), set(nasari_bs1), set(nasari_bs2)) 
synsets_syninoms = get_synonims_for_synsets(synsets)

#### Output

In [101]:
paolo_list = []
davide_list = []
for i in range(50):
    paolo_list.append([terms_1[i], terms_2[i], bs1_p[i], bs2_p[i], synsets_syninoms[bs1_p[i]], synsets_syninoms[bs2_p[i]]])
    davide_list.append([terms_1[i], terms_2[i], bs1_d[i], bs2_d[i], synsets_syninoms[bs1_d[i]], synsets_syninoms[bs2_d[i]]])

In [102]:
paolo_list[5]

['partito politico', 'associazione', 'bn:00060834n', 'bn:14146654n', ['forza_politica', 'gruppi_politici', 'Storia_dei_partiti_politici', 'movimento_politico', 'parte', 'partito_politico', 'politica_di_partito', 'partito', "L'origine_dei_partiti", 'partiti_politici', 'appartenenza_politica', 'partiti'], ['ordine', 'club', 'associazione', 'lega']]

In [103]:
davide_list[5]

['partito politico', 'associazione', 'bn:00060834n', 'bn:00006539n', ['forza_politica', 'gruppi_politici', 'Storia_dei_partiti_politici', 'movimento_politico', 'parte', 'partito_politico', 'politica_di_partito', 'partito', "L'origine_dei_partiti", 'partiti_politici', 'appartenenza_politica', 'partiti'], ['organizzazione_di_volontariato', 'classe', 'consorzio', 'ordine', 'associazioni_non_riconosciute', 'associazione_(diritto)', 'associazione_non_riconosciuta', 'associazione_di_volontariato', 'associazioni_prive_di_personalità_giuridica', 'associazione_senza_personalità_giuridica', 'associazione', 'corporazione', 'consociazione', 'categoria']]

#### Output pretty print

##### Stampa Paolo

In [105]:
for el in paolo_list:
    print(el[0] + "\t" + el[1] + "\t" + el[2] + "\t" + el[3])
    print(','.join(el[4][:3]) + "\t" + ','.join(el[5][:3]) + "\n")

biotopo	biologia	bn:03353031n	bn:00010543n
biotopo,biotopi	scienze_della_vita,biologico,scienza_biologica

magma	vulcano	bn:00052703n	bn:00079748n
magmatica,magmatico,differenziazione_magmatica	vulcano

brainstorming	telescopio	bn:00012707n	bn:00069738n
brainstorming,il_brainstorming,Brain_storming	cannocchiale,telescopio,montature_per_telescopi

livello	punteggio	bn:00025965n	bn:00041241n
grado_evolutivo,grado,misura	votazione,grado,voto

centesimo	affare	bn:00017162n	bn:00014152n
centime,centesimi,centesimo	offerta_impresa,accordo,affare

partito politico	associazione	bn:00060834n	bn:14146654n
forza_politica,gruppi_politici,Storia_dei_partiti_politici	ordine,club,associazione

tsunami	mare	bn:00078509n	bn:00069946n
onda_anomala,maremoti,Zunami	acque,mare_interno,mare

struzzo	frutteto	bn:00059688n	bn:00041967n
struzzo,struthio_camelus,struzzi	piantagione_di_zucchero,frutteti,piantagione_di_caffè

cannella	caramella	bn:00019142n	bn:00015227n
cannella,cinnamomo	caramella,caramelle,dolc

##### Stampa Davide

In [107]:
for el in davide_list:
    print(el[0] + "\t" + el[1] + "\t" + el[2] + "\t" + el[3])
    print(','.join(el[4][:3]) + "\t" + ','.join(el[5][:3]) + "\n")

biotopo	biologia	bn:03353031n	bn:00010543n
biotopo,biotopi	scienze_della_vita,biologico,scienza_biologica

magma	vulcano	bn:00052703n	bn:00079748n
magmatica,magmatico,differenziazione_magmatica	vulcano

brainstorming	telescopio	bn:00012707n	bn:00069738n
brainstorming,il_brainstorming,Brain_storming	cannocchiale,telescopio,montature_per_telescopi

livello	punteggio	bn:00025965n	bn:00041241n
grado_evolutivo,grado,misura	votazione,grado,voto

centesimo	affare	bn:00017162n	bn:00014152n
centime,centesimi,centesimo	offerta_impresa,accordo,affare

partito politico	associazione	bn:00060834n	bn:00006539n
forza_politica,gruppi_politici,Storia_dei_partiti_politici	organizzazione_di_volontariato,classe,consorzio

tsunami	mare	bn:00078509n	bn:00069946n
onda_anomala,maremoti,Zunami	acque,mare_interno,mare

struzzo	frutteto	bn:00059688n	bn:00041967n
struzzo,struthio_camelus,struzzi	piantagione_di_zucchero,frutteti,piantagione_di_caffè

cannella	caramella	bn:00019142n	bn:00021707n
cannella,cinnamomo	c

### Kappa di Cohen

##### Synset 1 a confronto

In [109]:
cohen_kappa_score(bs1_p, bs1_d)

0.67558799675588

##### Synset 2 a confronto

In [111]:
cohen_kappa_score(bs2_p, bs2_d)

0.635036496350365

### Valutazione annotazione: Nostra vs NASARI

Contiamo quante volte abbiamo annotato un sysnet diverso rispetto a quello trovato massimizzando la cosine similarity nel file mini_NASARI

In [119]:
def eval_annotation(syns_list1, syns_list_2):
    ''' Return a list with the position of the different annotations '''
    diff = []
    for i in range(len(syns_list1)):
        if(syns_list1[i] != syns_list_2[i]):
            diff.append(i)
    return diff

def eval_annotation_couple(syns_list1, syns_list_2, syns_list_1a, syns_list_2a):
    ''' Return a list with the position of the different annotations '''
    diff = []
    for i in range(len(syns_list1)):
        if(syns_list1[i] != syns_list_1a[i] or syns_list_2[i] != syns_list_2a[i]):
            diff.append(i)
    return diff

##### Syns_1 Paolo vs Nasari

In [114]:
len(eval_annotation(bs1_p, nasari_bs1))

28

##### Syns_1 Davide vs Nasari

In [115]:
len(eval_annotation(bs1_d, nasari_bs1))

24

##### Syns_2 Paolo vs Nasari

In [116]:
len(eval_annotation(bs2_p, nasari_bs2))

30

##### Syns_2 Davide vs Nasari

In [117]:
len(eval_annotation(bs2_d, nasari_bs2))

28

#### Evaluation coppie

##### Paolo vs NASARI

In [124]:
len(eval_annotation_couple(bs1_p, bs2_p, nasari_bs1, nasari_bs2))

35

##### Davide vs NASARI

In [125]:
len(eval_annotation_couple(bs1_d, bs2_d, nasari_bs1, nasari_bs2))

36